# <span style="color: red;">Создаём графический интерфей пользователя</span>

Делал по статье ["PyQt6 — полное руководство для новичков"](https://habr.com/ru/companies/skillfactory/articles/599599/)

In [ ]:
# Установка PyQt:
# !pip install pyqt6
# и на будущее
# !pip install pyqt-tools

# импортируем классы PyQt для приложения:
# QApplication - это обработчик приложения
# QWidget - базовый пустой виджет графического интерфейса (оба из модуля QtWidgets)
from PyQt6.QtWidgets import (
    QApplication, QMainWindow, QFileDialog, QTextEdit, QVBoxLayout, QHBoxLayout, QWidget
)
from PyQt6.QtGui import QAction
from PyQt6.QtCore import QSize

from matplotlib.backends.backend_qtagg import NavigationToolbar2QT as NavigationToolbar

import sys # Только для доступа к аргументам командной строки


# мои функции
from open_csv_file import open_csv_file

# Для корректной работы PyQt в Jupyter
%gui qt

import sys

class QtOutput:
    def __init__(self, textedit):
        self.textedit = textedit
    def write(self, msg):
        # Добавляем текст в конец QTextEdit
        cursor = self.textedit.textCursor()
        cursor.movePosition(cursor.MoveOperation.End)
        self.textedit.setTextCursor(cursor)
        self.textedit.insertPlainText(str(msg))
    def flush(self):
        pass

# Подкласс QMainWindow для настройки главного окна приложения
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # self.format_ver = 1 # версия формата CSV файла
        # self.inx_start = 0 # начальный индекс для отображения
        # self.inx_stop = None # конечный индекс для отображения
        # self.downsampling_factor = 100 # коэффициент даунсемплинга

        self.setWindowTitle("OscViwer")
        self.setMinimumSize(QSize(800, 500))

        # Текстовое поле для сообщений
        self.status_text = QTextEdit()
        self.status_text.setReadOnly(True)

        # Виджет для графика
        self.plot_widget = QWidget()

        # # Layout: горизонтально — слева график, справа сообщения
        # layout = QHBoxLayout()
        # layout.addWidget(self.plot_widget, 3)
        # layout.addWidget(self.status_text, 1)
        # central_widget = QWidget()
        # central_widget.setLayout(layout)
        # self.setCentralWidget(central_widget)

        # Layout: горизонтально — слева сообщения, справа график
        layout = QVBoxLayout()
        layout.addWidget(self.plot_widget, 3)
        layout.addWidget(self.status_text, 1)
        central_widget = QWidget()
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        # Меню
        menubar = self.menuBar()
        file_menu = menubar.addMenu("Файл")
        # пункт меню "Открыть" (открыть CSV файл)
        open_action = QAction("Открыть CSV...", self)
        open_action.triggered.connect(lambda: open_csv_file(self))
        file_menu.addAction(open_action)

        self._old_stdout = sys.stdout
        # Перенаправляем stdout на QtOutput
        sys.stdout = self._old_stdout
        sys.stdout = QtOutput(self.status_text)

    def show_message(self, text):
        self.status_text.append(text)

    def closeEvent(self, event):
        # стандартный вывод будет возвращён обратно в терминал
        sys.stdout = self._old_stdout
        super().closeEvent(event)

# Запуск приложения (вне Jupyter)
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    app.exec()

## [open_csv_file](open_csv_file.py)

open_csv_file(main_window)

Функция open_csv_file открывает диалог выбора CSV-файла, восстанавливая последнюю открытую директорию из ini-файла (хранится как JSON рядом со скриптом). Сначала пытается прочитать путь last_dir из osc_viewer.ini; если это не удаётся, используется пустая строка. Затем показывается QFileDialog с фильтром по CSV. После выбора файла функция сохраняет директорию выбранного файла обратно в ini, чтобы в следующий раз открыть тот же путь.

Далее инициализируются параметры по умолчанию в main_window: версия формата CSV (format_ver = 1), индексы диапазона для отображения (inx_start = 0, inx_stop = None) и коэффициент прореживания (downsampling_factor = 1). Пользователю выводится сообщение о выбранном файле. Параллельно функция пытается загрузить файл параметров с тем же именем, но с расширением .json (рядом с CSV): если он есть, параметры из него подставляются в main_window и сообщаются пользователю; если JSON отсутствует, остаются значения по умолчанию, и это явно сообщается.

После подготовки параметров вызывается load_and_prepare_data, которому передаются имя файла и актуальные значения format_ver, inx_start, inx_stop и downsampling_factor. Возвращённые временной ряд t и сигнал s передаются в plot_signal_to_qt для отрисовки. Параметр add_mode=True указывает рисовать новый график поверх уже существующего, не очищая область построения. Все возможные ошибки оборачиваются в общий try/except: при исключении пользователю показывается текст ошибки через main_window.show_message.

## [load_and_prepare_data](load_and_prepare_data.py)

def load_and_prepare_data(file_name, format_ver, inx_start, inx_stop, downsampling_factor)

Функция загружает измеренные данные из текстового CSV-файла, нормализует их и подготавливает для дальнейшего анализа (например, для БПФ). На выходе она возвращает временной ряд t, отсчёты сигнала s, метаданные в двух видах, шаг дискретизации dt, коэффициент «оверсэмплинга» и исходную длину окна N.

Входной файл читается построчно и парсится по формату. Для format_ver = 0 строки с непустыми первыми тремя полями трактуются как метаданные (пара ключ–значение), а строки с пустыми тремя первыми полями — как данные: время берётся из столбца 4, сигнал — из столбца 5. Для format_ver = 1 первая строка — заголовки, вторая — значения метаданных; среди них ищется ключ Increment, задающий шаг по времени $k_t$ (сек/отсчёт). Начиная с третьей строки читаются данные: первый столбец интерпретируется как индекс или время, умножается на $k_t$ для получения секунд, второй — значение сигнала. Пустые строки пропускаются. После чтения выводится количество считанных отсчётов и оценка исходной частоты дискретизации $F_s = 1/k_t$ в МГц.

Затем выбирается рабочий интервал [inx_start:inx_stop] и, при необходимости, применяется прореживание с шагом downsampling_factor. Временная ось сдвигается так, чтобы начинаться с нуля. Сигнал центрируется вычитанием среднего для устранения DC-смещения.

Из словаря метаданных формируется объект MetaInfo и DataFrame meta_df с колонками Key/Value для удобного доступа (например, извлекается возможный сдвиг времени Start). Далее вычисляется размер окна N и коэффициент oversampling_factor = $2^{16}/N$. На его основании целевая длина N_new выбирается как $N_{new} = \lfloor \text{oversampling\_factor} \cdot N \rfloor$, что при таком определении даёт $N_{new} = 2^{16}$ практически всегда (то есть выполняется нулевое дополнение до 65536 отсчётов, если исходная длина меньше). Сигнал s дополняется нулями до N_new. Временной шаг $dt = t[1]-t[0]$, временная ось t расширяется синхронно до той же длины равномерной сеткой, после чего ко всем меткам времени добавляется сдвиг Start (если задан). Итоговая частота дискретизации пересчитывается и выводится как $F'_s = 1/dt$ в МГц.

Возвращаемые значения: список времён t (с возможным сдвигом и расширением), список отсчётов s (с нулевым дополнением), объект метаданных meta_info, таблица метаданных meta_df, шаг дискретизации dt, коэффициент oversampling_factor, а также N — исходная длина после обрезки/прореживания (до дополнения нулями).

Замечания и потенциальные «грабли»:
- Для format_ver = 0 переменная $k_t$ остаётся 1.0, поэтому печатаемая «исходная» частота дискретизации $1/k_t$ может быть неинформативна. Надёжнее выводить частоту из $dt$, когда он уже известен.
- Если в данных менее двух точек, обращения к t[1] и t[0] вызовут ошибку. Нужны проверки на минимальную длину.
- Выбор индексов и прореживание используют t[inx_start] для выравнивания нуля — это корректно, но стоит проверять, что срез не пуст и индексы валидны.
- oversampling_factor по сути отражает коэффициент дополнения до длины $2^{16}$, а не интерполяцию; возможно, уместнее название вроде pad_factor или fft_pad_factor.
- Возвращаемое N — длина до дополнения, тогда как t и s уже могут иметь длину N_new. Это может путать потребителей функции; разумно вернуть также N_new или заменить N на фактическую длину.
- Чтение CSV вручную уязвимо к форматным отклонениям; при возможности можно задействовать pandas.read_csv с явными именами столбцов для format_ver = 1.

## [plot_to_qt](plot_to_qt.py)

Функция строит график осциллограммы с помощью Matplotlib внутри произвольного Qt-виджета и умеет добавлять новые кривые поверх уже отрисованных. На вход подаются массивы времени и сигнала; время сразу переводится в миллисекунды (умножением на 1000), чтобы ось X была в удобных единицах. Далее функция гарантирует, что у контейнерного виджета есть QVBoxLayout: если его нет, создаёт и назначает. В зависимости от флага add_mode либо очищает область построения (полностью удаляя из layout предыдущие виджеты), затем создаёт новую пару FigureCanvas + NavigationToolbar и ось ax, либо пытается найти уже существующий FigureCanvas и переиспользовать его (в этом случае ось берётся как текущая у фигуры). После подготовки канвы данные добавляются вызовом ax.plot(t_ms, s_arr).

К графику привязывается обработчик событий клавиатуры, чтобы управлять «активной» кривой. Активной по умолчанию становится последняя добавленная линия. Клавиши Left/< и Right/> сдвигают только активный график по оси X на 1% текущего видимого диапазона: фактически изменяется массив x у Line2D, затем вызываются relim и autoscale_view для пересчёта границ и перерисовка. Пробел переключает активную кривую по кругу среди ax.lines и визуально подчёркивает её (большая толщина, выше z-order, непрозрачность 1.0), а остальные ослабляются (меньше толщина, ниже z-order, alpha 0.7). Чтобы не плодить дубликаты обработчиков, функция хранит id последней подписки на canvas и при повторном вызове сначала отключает прежний обработчик, затем подписывается заново. Также канве выдаётся сильная фокусировка, чтобы она принимала события клавиатуры.

В конце оформляются подписи осей, заголовок и сетка. Если данные непустые, ось X инициализируется диапазоном от минимального времени до последнего значения (предполагается, что время отсортировано по возрастанию). Выполняется принудительная перерисовка canvas.draw(), чтобы отразить изменения на экране.

Нюансы и осторожности. Сдвиг по стрелкам изменяет сами данные линии (xdata), а не окно просмотра; это поведение намеренно «сдвигает» кривую относительно других, но если нужно панорамирование области, лучше менять xlim оси. При add_mode=True повторное получение оси через fig.gca() может быть не лучшей практикой на новых версиях Matplotlib — надёжнее явно хранить/забирать ax из фигуры (например, fig.axes[0]). Двойной вызов setFocus избыточен. Инициализация пределов X через min(t_ms) и t_ms[-1] корректна для монотонно растущего времени; если временная ось была модифицирована (сдвинута) до вызова, границы можно задавать по np.min/np.max.